In [3]:
using BenchmarkTools
using ForneyLab

ArgumentError: ArgumentError: Package ForneyLab not found in current path:
- Run `import Pkg; Pkg.add("ForneyLab")` to install the ForneyLab package.


In [2]:
@btime begin 
    # Data
    n_samples = 1000
    x = [ t for t = 1:n_samples ] # True state
    y = x + sqrt(200.0) * randn(n_samples); # Noisy observations of state 
    
    g = FactorGraph()

    # declare priors as random variables
    @RV m_x_t_min # m_x_t_min = Variable(id=:m_x_t_min)
    @RV v_x_t_min # v_x_t_min = Variable(id=:v_x_t_min)
    @RV x_t_min ~ GaussianMeanVariance(m_x_t_min, v_x_t_min)

    # System equations
    # u = 1.0; v = 200.0
    @RV n_t ~ GaussianMeanVariance(0.0, 200.0)
    @RV x_t = x_t_min + 1.0
    @RV y_t = x_t + n_t

    # Name variable for ease of lookup
    #x_t.id = :x_t;
    
    # Placeholders for prior
    placeholder(m_x_t_min, :m_x_t_min) # placeholder(:m_x_t_min) does not work
    placeholder(v_x_t_min, :v_x_t_min)

    # Placeholder for data
    placeholder(y_t, :y_t);
    
    algo = sumProductAlgorithm(x_t) # Figure out a schedule and compile to Julia code
    
    # Define algorithm
    eval(Meta.parse(algo))
    
    # Define values for prior statistics
    m_x_0 = 0.0
    v_x_0 = 1000.0

    m_x = Vector{Float64}(undef, n_samples)

    m_x_t_min = m_x_0
    v_x_t_min = v_x_0
    for t = 1:n_samples
        # Prepare data and prior statistics
        data = Dict(:y_t       => y[t],
                    :m_x_t_min => m_x_t_min,
                    :v_x_t_min => v_x_t_min)

        # Execute algorithm
        marginals = step!(data)

        # Extract posterior statistics
        m_x_t_min = mean(marginals[:x_t])
        v_x_t_min = var(marginals[:x_t])

        # Store to buffer 
        m_x[t] = m_x_t_min
    end
    
    return m_x
end;

LoadError: UndefVarError: @RV not defined